# Process VODCA v2

Downloaded from here: https://essd.copernicus.org/preprints/essd-2024-35/


## Load packages

In [ ]:
import sys
import os
import xarray as xr
import pandas as pd
import numpy as np
from odc.geo.xr import assign_crs

## Combine files

In [ ]:
base='/g/data/os22/chad_tmp/Aus_phenology/data/VODCA_CXKu/daily_images_VODCA_CXKu/'
years = [str(i) for i in range(1988,2022)]

In [ ]:
dss=[]
for y in years:
    print(y)
    files = [base+y+'/'+i for i in os.listdir(base+y)]
    files.sort()
    ds = xr.open_mfdataset(files)
    ds = ds.rename({'lat':'latitude', 'lon':'longitude'})
    ds = assign_crs(ds['VODCA_CXKu'], crs='EPSG:4326')
    ds = ds.sel(latitude=slice(-10,-45), longitude=slice(110,155))
    ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
    dss.append(ds.compute())

vodca = xr.concat(dss, dim='time').sortby('time')

In [ ]:
vodca.mean('time').plot(robust=True)

In [ ]:
vodca.mean(['latitude','longitude']).plot(figsize=(12,4))

In [ ]:
vodca.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/VODCAv2_1988_2021.nc')

## Process TerraClimate PET

And calculate P:PET

Other datasets: http://thredds.northwestknowledge.net:8080/thredds/terraclimate_aggregated.html

In [ ]:
from _utils import round_coords

In [ ]:
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import start_local_dask, round_coords
client = start_local_dask()
client

In [ ]:
ds = xr.open_dataset('http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_terraclimate_pet_1958_CurrentYear_GLOBE.nc')
ds = ds.rename({'lat':'latitude', 'lon':'longitude'})
ds = assign_crs(ds['pet'], crs='EPSG:4326')
ds = ds.sel(latitude=slice(-10,-45), longitude=slice(110,155))
ds = ds.sel(time=slice('1981','2022'))

rain = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/rain_5km_monthly_1981_2022.nc')['rain']
ds['time'] = rain.time #shift time by half month
ds.attrs['nodata']=np.nan
ds = ds.chunk(time=1)
ds = ds.odc.reproject(how=rain.odc.geobox, resampling='bilinear').compute()
ds = round_coords(ds)
ds = ds.rename('PET')
ds.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/PET_1981_2022.nc')

In [ ]:
mi = rain/ds
mi = mi.rename('MI')

In [ ]:
mi.sel(time=slice('1982','2022')).to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/MI_1982_2022.nc')